In [21]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

import pathlib


In [2]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [3]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [45]:
def load_model():
  model_dir = './'
  model = tf.saved_model.load(model_dir)

  return model

In [46]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './saved_model.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [47]:
detection_model = load_model()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [50]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [51]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [60]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('./images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('images/Chiba_20170913094022.jpg'),
 PosixPath('images/Chiba_20170913094025.jpg'),
 PosixPath('images/Chiba_20170913095152.jpg'),
 PosixPath('images/Chiba_20170913095454.jpg'),
 PosixPath('images/Chiba_20170913095458.jpg'),
 PosixPath('images/Chiba_20170913095519.jpg'),
 PosixPath('images/Chiba_20170913100451.jpg'),
 PosixPath('images/Chiba_20170913100800.jpg'),
 PosixPath('images/Chiba_20170913101057.jpg'),
 PosixPath('images/Chiba_20170913101100.jpg'),
 PosixPath('images/Chiba_20170913101103.jpg'),
 PosixPath('images/Chiba_20170913101119.jpg'),
 PosixPath('images/Chiba_20170913101123.jpg'),
 PosixPath('images/Chiba_20170913101253.jpg'),
 PosixPath('images/Chiba_20170913101312.jpg'),
 PosixPath('images/Chiba_20170913102209.jpg'),
 PosixPath('images/Chiba_20170913102738.jpg'),
 PosixPath('images/Chiba_20170913102739.jpg'),
 PosixPath('images/Chiba_20170913102740.jpg'),
 PosixPath('images/Chiba_20170913103458.jpg'),
 PosixPath('images/Chiba_20170913103824.jpg'),
 PosixPath('i

In [61]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)


TypeError: 'AutoTrackable' object is not callable

In [62]:
print(detection_model)

In [65]:

import numpy as np
import os
import tensorflow as tf
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# path to the frozen graph:
PATH_TO_FROZEN_GRAPH = './saved_model.pb'

# path to the label map
PATH_TO_LABEL_MAP = './saved_model.pbtxt'

# number of classes 
NUM_CLASSES = 2

cap = cv2.VideoCapture(0)

#reads the frozen graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABEL_MAP)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Detection
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while True:
            # Read frame from camera
            ret, image_np = cap.read()
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Extract image tensor
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Extract detection boxes
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Extract detection scores
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            # Extract detection classes
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            # Extract number of detections
            num_detections = detection_graph.get_tensor_by_name(
                'num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=3,
                )
        # Display output
            cv2.imshow('Gun Detection', cv2.resize(image_np, (1200, 800)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break



AttributeError: module 'tensorflow' has no attribute 'GraphDef'

In [67]:
import tensorflow as tf
import cv2

img = cv2.imread("./images/Chiba_20170913094022.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_rgb = np.expand_dims(img, 0)

predict_fn = tf.contrib.predictor.from_saved_model("./saved_model")
output_data = predict_fn({"inputs": img_rgb})
print(output_data)  # detector output dictionary

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [ ]:
def predict(x):
  example = tf.train.Example()
  example.features.feature["x"].float_list.value.extend([x])
  return detection_model.signatures["predict"](examples=tf.constant([example.SerializeToString()]))

print(predict(1))
print(predict(3.5))